In [1]:
import pandas as pd

In [70]:
df = pd.read_json('data/data.json')

In [71]:
#needed to verify & create currency match column
Country_dictionary= {'IE': 'EUR', 'FR': 'EUR', 'ES': 'EUR', 'NL': 'EUR', 'DE': 'EUR', 'BE': 'EUR', 'AT': 'EUR', 'PT': 'EUR', 'IT': 'EUR', 'HU': 'EUR', 'RO': 'EUR', 'CZ': 'EUR', 'FI': 'EUR', 'HR': 'EUR', 'BG': 'EUR', 'SI': 'EUR', 'SE': 'EUR', 'DK': 'EUR', 'US': 'USD', 'CA': 'CAD', 'GB': 'GBP', 'AU': 'AUD', 'NZ': 'NZD', 'MX': 'MXN'}
#needed for drop all unneeded columns
DROP = ['approx_payout_date', 'body_length', 
        'description','email_domain',
       'event_created', 'event_end', 'event_published', 'event_start',
       'has_header',
       'listed', 'name', 'name_length', 'num_payouts',
       'object_id', 'org_desc', 'org_facebook', 'org_name', 'org_twitter',
       'payee_name', 'sale_duration',
       'sale_duration2',  'ticket_types', 
       'user_created', 'user_type', 'venue_address', 'venue_country',
       'venue_latitude', 'venue_longitude', 'venue_name', 'venue_state','payout_type','previous_payouts',
       'country','currency', 'length_before_event', 'event_published']

In [73]:
fraud = ['fraudster_event','spammer_warn', 'fraudster',
       'spammer_limited', 'spammer_noinvite', 'locked', 'tos_lock',
       'tos_warn', 'fraudster_att', 'spammer_web', 'spammer']

In [74]:
df['acct_type'].replace(fraud,1,inplace=True)
df['acct_type'].replace('premium',0,inplace=True)

In [75]:
#Needed new Columns
## Clean Code
df['event_length'] = df['event_end'] - df['event_start']
df['length_before_event'] = df['event_published'] - df['event_created']
df['event_published_present'] = df['event_published']
df['event_published_present'] = df['event_published_present'].isna()
df['event_published_present'].replace(to_replace=[False, True], value=[2,3], inplace=True)
df['event_published_present'].replace(to_replace=[2,3], value=[1,0], inplace=True)

In [76]:
df['payout_type'][df['payout_type'] == 'ACH'] = 'ACH'
df['payout_type'][df['payout_type'] == 'CHECK'] = 'CHECK'
df['payout_type'][df['payout_type'] == ''] = 'No payout_type listed'
#new previous payouts
df['num_previous_payouts'] = list(map(len, df['previous_payouts']))

/home/pipingkid/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/pipingkid/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/pipingkid/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing 

In [77]:
#inplace methods
df['country'].replace('', None,inplace = True)
df['delivery_method'].fillna(value=4,inplace=True)
df['currency_match']=(df['country'].map(Country_dictionary) == df['currency']).replace([True,False],[1,0]) 
df['venue_name_exits'] = df['venue_name'].isnull().replace([False, True], [0,1])       


In [78]:
#hard code dummies
df['email_gmail'] = 0
df['email_gmail'][df['email_domain'].str.contains('gmail')] = 1
df['email_yahoo'] = 0
df['email_yahoo'][df['email_domain'].str.contains('yahoo')] = 1
df['email_aol'] = 0
df['email_aol'][df['email_domain'].str.contains('aol')] = 1
df['email_hotmail'] = 0
df['email_hotmail'][df['email_domain'].str.contains('hotmail')] = 1
# user age listed dummy variable
df['user_age_listed'] = True
df['user_age_listed'][df['user_age'] == 0] = False

/home/pipingkid/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/pipingkid/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/pipingkid/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/pipingkid/anac

In [79]:
#get dummies
country_dummies = pd.get_dummies(df['country'])
currency_dummies = pd.get_dummies(df['currency'])
delivery_dummy = pd.get_dummies(df['delivery_method'])
payout_type = pd.get_dummies(df['payout_type'])


In [80]:
#dummy variables to concate at end
dummy_add = [df, country_dummies, currency_dummies, delivery_dummy, payout_type]

In [81]:
#Drop Everything We Don't Need -- CAREFUL
df.drop(DROP,axis = 1,inplace=True) 

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14337 entries, 0 to 14336
Data columns (total 20 columns):
acct_type                  14337 non-null int64
channels                   14337 non-null int64
delivery_method            14337 non-null float64
fb_published               14337 non-null int64
gts                        14337 non-null float64
has_analytics              14337 non-null int64
has_logo                   14337 non-null int64
num_order                  14337 non-null int64
show_map                   14337 non-null int64
user_age                   14337 non-null int64
event_length               14337 non-null int64
event_published_present    14337 non-null int64
num_previous_payouts       14337 non-null int64
currency_match             14337 non-null int64
venue_name_exits           14337 non-null int64
email_gmail                14337 non-null int64
email_yahoo                14337 non-null int64
email_aol                  14337 non-null int64
email_hotmail          

In [83]:
#add it all together
final_df = pd.concat(dummy_add, axis=1)

In [84]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14337 entries, 0 to 14336
Columns: 105 entries, acct_type to No payout_type listed
dtypes: bool(1), float64(2), int64(17), uint8(85)
memory usage: 3.3 MB


In [86]:
final_df.to_csv('data/cleaned_data.csv')

In [89]:
print(final_df.columns)

Index([            'acct_type',              'channels',
             'delivery_method',          'fb_published',
                         'gts',         'has_analytics',
                    'has_logo',             'num_order',
                    'show_map',              'user_age',
       ...
                         'MXN',                   'NZD',
                         'USD',                     0.0,
                           1.0,                     3.0,
                           4.0,                   'ACH',
                       'CHECK', 'No payout_type listed'],
      dtype='object', length=105)
